# 人机协同工具验证

本教程演示如何为任何工具添加人工验证。我们将使用`HumanApprovalCallbackhandler`来实现这一点。

假设我们需要使用`ShellTool`。将此工具添加到自动化流程中会带来明显的风险。让我们看看如何强制对输入进入此工具进行手动人工批准。

**注意**：我们通常不建议使用`ShellTool`。有很多误用的方式，并且大多数情况下不需要使用它。我们在这里仅用于演示目的。

In [1]:
# 导入 HumanApprovalCallbackHandler 类
from langchain.callbacks import HumanApprovalCallbackHandler
# 导入 ShellTool 类
from langchain.tools import ShellTool

In [19]:
# 创建ShellTool对象
tool = ShellTool()

In [20]:
# 调用tool模块中的run函数，并传入参数"echo Hello World!"，将返回值打印出来
print(tool.run("echo Hello World!"))

Hello World!



## 添加人工批准
将默认的`HumanApprovalCallbackHandler`添加到工具中，这样用户就必须在命令实际执行之前手动批准工具的每个输入。

In [10]:
# 创建一个ShellTool对象，并传入一个HumanApprovalCallbackHandler对象作为参数
tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])

In [15]:
# 调用 tool 对象的 run 方法，并传入参数 "ls /usr"，该方法会执行系统命令并返回结果
print(tool.run("ls /usr"))

Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

ls /usr
yes
X11
X11R6
bin
lib
libexec
local
sbin
share
standalone



In [17]:
# 调用tool对象的run方法，并传入参数"ls /private"，该方法会执行系统命令"ls /private"
print(tool.run("ls /private"))

Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

ls /private
no


HumanRejectedException: Inputs ls /private to tool {'name': 'terminal', 'description': 'Run shell commands on this MacOS machine.'} were rejected.

## 配置人工审批

假设我们有一个代理程序，它接收多个工具，并且我们希望它只在某些工具和某些输入上触发人工审批请求。我们可以配置我们的回调处理程序来实现这一点。

In [ ]:
# 导入所需的模块
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

In [33]:
def _should_check(serialized_obj: dict) -> bool:
    # 仅对ShellTool需要批准。
    return serialized_obj.get("name") == "terminal"


def _approve(_input: str) -> bool:
    if _input == "echo 'Hello World'":
        return True
    msg = (
        "您是否批准以下输入？"
        "除了'Y'/'Yes'（不区分大小写）之外的任何内容都将被视为否定。"
    )
    msg += "\n\n" + _input + "\n"
    resp = input(msg)
    return resp.lower() in ("yes", "y")


callbacks = [HumanApprovalCallbackHandler(should_check=_should_check, approve=_approve)]

In [34]:
# 创建一个OpenAI对象，设置temperature参数为0
llm = OpenAI(temperature=0)

# 载入所需的工具，包括"wikipedia", "llm-math", "terminal"，并传入llm对象
tools = load_tools(["wikipedia", "llm-math", "terminal"], llm=llm)

# 初始化一个agent，传入tools、llm对象以及agent类型AgentType.ZERO_SHOT_REACT_DESCRIPTION
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [38]:
# 定义问题
question = "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany."

# 运行代理程序，并传入问题和回调函数
agent.run(question, callbacks=callbacks)

'Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago.'

In [36]:

# 定义要运行的代码
code = "print 'Hello World' in the terminal"

# 运行代码，并指定回调函数
agent.run(code, callbacks=callbacks)

'Hello World'

In [39]:
# 运行代理程序，列出 /private 目录中的所有子目录
agent.run("list all directories in /private", callbacks=callbacks)

Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

ls /private
no


HumanRejectedException: Inputs ls /private to tool {'name': 'terminal', 'description': 'Run shell commands on this MacOS machine.'} were rejected.